In [246]:
import pandas as pd
import json
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime, timedelta
import requests

In [247]:
from bs4 import BeautifulSoup

# generate trip id from h2


def get_id(soup):
    trip = []
    header = soup.find_all(["h2"])
    print("Number of headers :" + str(len(header)))
    for h2s in header:
        # print(h2s)
        num = re.search(r"\d{9}", h2s.text)
        if num:
            trip.append(num.group())
    # print(trip)
    print("Number of Trips :" + str(len(trip)))


# checking url
url = "http://www.psudataeng.com:8000/getStopEvents"
html = urlopen(url)
# response = requests.get(url)
# html = response.content
soup = BeautifulSoup(html, "lxml")
print("Reading URL..")
get_id(soup)

Reading URL..
Number of headers :580
Number of Trips :580


In [248]:
trip = []
file_path = "/Users/vysalikallepalli/Desktop/CS510_DataEng_Project/Part-3/test/stopevents_2023-05-25.html"
with open(file_path, "r") as html_file:
    soup = BeautifulSoup(html_file, "lxml")
    print("Reading File..")
header = soup.find_all(["h2"])
print("Number of headers :" + str(len(header)))
for h2s in header:
    # print(h2s)
    num = re.search(r"\d+", h2s.text)
    if num:
        trip.append(num.group())
    # print(trip)
print("Number of Trips :" + str(len(trip)))
# extract table data
stopdata = []
tables = soup.find_all("table")
length = len(tables)
for i in range(length):
    data = tables[i].find_all("td")
    dict = {}
    dict["trip_id"] = trip[i]
    dict["vehicle_id"] = data[0].string
    dict["route_id"] = data[3].string
    dict["direction"] = data[4].string
    dict["service_key"] = data[5].string
    stopdata.append(dict)
    service_keys = {data['service_key'] for data in stopdata}
    if service_keys:
        index = list(service_keys)[0]
        for data in stopdata:
            data["service_key"] = index

print(len(stopdata))

Reading File..
Number of headers :278
Number of Trips :278
278


In [249]:
df = pd.DataFrame(stopdata)
df_check = df.loc[df["service_key"] != 'M']
df_check

,trip_id,vehicle_id,route_id,direction,service_key


In [250]:
df

,trip_id,vehicle_id,route_id,direction,service_key
0,238867474,3141,16,0,M
1,238867533,3141,16,1,M
2,238867580,3141,16,0,M
3,238867634,3141,16,1,M
4,238867687,3141,16,0,M
...,...,...,...,...,...
273,239376440,3960,9,0,M
274,239376538,3960,9,1,M
275,239376644,3960,9,0,M
276,239376748,3960,9,1,M


In [251]:
assertion1 = "Route id exists and is a positive number"
assert "route_id" in df.columns and df["route_id"].all(
) >= 0, f"{assertion1}: NOT MET"
print(f"{assertion1} : MET")

Route id exists and is a positive number : MET


In [252]:
assertion2 = "The service key represents the day of the week, and it can take one of three possible values: M,S,W,U"
assert df["service_key"].isin(['M', 'S', 'W', 'U']).all(), f"{assertion2}: NOT MET"
f"{assertion2} : MET"

'The service key represents the day of the week, and it can take one of three possible values: M,S,W,U : MET'

In [253]:
assertion3 = "Trip id a non negative, 9-digit integer"
assert (
    df["trip_id"].all() > 0 & df["trip_id"].astype(int).between(99999999, 10000000000).all()
    ), f"{assertion3}: NOT MET"
print(f"{assertion3} : MET")

Trip id a non negative, 9-digit integer : MET


In [254]:
assertion4 = "Direction exists and is either 0 or 1"
assert df["direction"].all() in [0, 1], f"{assertion4}: NOT MET"
print(f"{assertion4} : MET")

Direction exists and is either 0 or 1 : MET


In [255]:
assertion5 = "Vehicle ID is non-null and positive number"
assert df["vehicle_id"].all() >= 0, f"{assertion5}: NOT MET"
print(f"{assertion5} : MET")

Vehicle ID is non-null and positive number : MET


In [256]:
# stopdata
service_keys = {data['service_key'] for data in stopdata}
print(service_keys)
index = list(service_keys)
# for index, value in enumerate(service_keys):
#     print(f"Index: {index}, Value: {value}")
index[0]
df['service_key'] = index[0]
df = df.loc[df["service_key"] != 'M']
df


{'M'}


,trip_id,vehicle_id,route_id,direction,service_key


In [257]:
date = datetime.today().strftime("%d-%m-%Y")
date

'28-05-2023'

In [258]:
filename = f"test_{date}.json"
with open(filename, "w") as json_file:
    json.dump(stopdata, json_file)
print("Data written to JSON file:", filename)

Data written to JSON file: test_28-05-2023.json


In [269]:
df = pd.read_json("test_25-05-2023.json")
# df = pd.read_json("test_2023-01-14.json")

df

,trip_id,vehicle_id,route_id,direction,service_key
0,238867474,3141,16,0,M
1,238867533,3141,16,1,M
2,238867580,3141,16,0,M
3,238867634,3141,16,1,M
4,238867687,3141,16,0,M
...,...,...,...,...,...
273,239376440,3960,9,0,M
274,239376538,3960,9,1,M
275,239376644,3960,9,0,M
276,239376748,3960,9,1,M


In [270]:
df

,trip_id,vehicle_id,route_id,direction,service_key
0,238867474,3141,16,0,M
1,238867533,3141,16,1,M
2,238867580,3141,16,0,M
3,238867634,3141,16,1,M
4,238867687,3141,16,0,M
...,...,...,...,...,...
273,239376440,3960,9,0,M
274,239376538,3960,9,1,M
275,239376644,3960,9,0,M
276,239376748,3960,9,1,M


In [271]:
df['service_key'] = df['service_key'].apply(lambda x: 'Saturday' if x == 'S' else("Sunday" if x == 'U' else 'Weekday'))
df

,trip_id,vehicle_id,route_id,direction,service_key
0,238867474,3141,16,0,Weekday
1,238867533,3141,16,1,Weekday
2,238867580,3141,16,0,Weekday
3,238867634,3141,16,1,Weekday
4,238867687,3141,16,0,Weekday
...,...,...,...,...,...
273,239376440,3960,9,0,Weekday
274,239376538,3960,9,1,Weekday
275,239376644,3960,9,0,Weekday
276,239376748,3960,9,1,Weekday


In [272]:
df['direction'] = df['direction'].apply(lambda x: 'Out' if x == 0 else("Back" if x == 1 else 'Unknown'))
df

,trip_id,vehicle_id,route_id,direction,service_key
0,238867474,3141,16,Out,Weekday
1,238867533,3141,16,Back,Weekday
2,238867580,3141,16,Out,Weekday
3,238867634,3141,16,Back,Weekday
4,238867687,3141,16,Out,Weekday
...,...,...,...,...,...
273,239376440,3960,9,Out,Weekday
274,239376538,3960,9,Back,Weekday
275,239376644,3960,9,Out,Weekday
276,239376748,3960,9,Back,Weekday
